In [40]:
# code for autoreload of modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import os
# Set your custom working DIR:
DIR = '/Users/jannispoltier/Documents/Studium/3. Semester/tabpfn_credit_codebase'
# DIR = r'C:\Users\u0148775\PycharmProjects\CreditScoring'
os.chdir(DIR)



In [42]:

from src.utils import set_random_seed
set_random_seed(0)

In [46]:
from src.utils import load_config
from pathlib import Path

# read dataconfig from CONFIG_DATA.yaml file:

# dataconfig = load_config('config\CONFIG_DATA.yaml')
# experimentconfig = load_config('config\CONFIG_EXPERIMENT.yaml')
# methodconfig = load_config('config\CONFIG_METHOD.yaml')
# evaluationconfig = load_config('config\CONFIG_EVALUATION.yaml')
dataconfig = load_config(Path('config') / 'CONFIG_DATA.yaml')
experimentconfig = load_config(Path('config') / 'CONFIG_EXPERIMENT.yaml')
methodconfig = load_config(Path('config') / 'CONFIG_METHOD.yaml')
evaluationconfig = load_config(Path('config') / 'CONFIG_EVALUATION.yaml')
tuningconfig = load_config(Path('config') / 'CONFIG_TUNING.yaml')
paramconfig = load_config(Path('config') / 'datasets' / 'OPTIMAL_HYPERPARAS.yaml')


In [47]:
from src.classes.experiment import Experiment

# Initialize experiment object
experiment = Experiment(dataconfig=dataconfig,
                        experimentconfig=experimentconfig,
                        methodconfig=methodconfig,
                        evaluationconfig=evaluationconfig,
                        tuningconfig=tuningconfig,
                        paramconfig=paramconfig)


Experiment created at:  2025-06-03-16-47
Task: pd
Dataset: 01_gmsc
CV splits:  3


In [48]:
experiment.run()

01_gmsc loaded
- Dataset has 150000 rows. Subsampling to 30000 rows.
01_gmsc preprocessed
x shape:  (30000, 10)
y shape:  (30000,)


Cross-validation loop::   0%|          | 0/3 [00:00<?, ?it/s][I 2025-06-03 16:47:56,623] A new study created in memory with name: no-name-5d8c9266-5aac-405f-8b5d-f30fec04bda9
[I 2025-06-03 16:47:56,805] Trial 0 finished with value: 0.8428295276390947 and parameters: {'n_estimators': 60, 'learning_rate': 0.0530352388639183}. Best is trial 0 with value: 0.8428295276390947.


- Omitting rows with missing values: 6042 rows left out
---- Tuning ab ----


[I 2025-06-03 16:47:57,124] Trial 1 finished with value: 0.8503842672776106 and parameters: {'n_estimators': 100, 'learning_rate': 0.09976662828086057}. Best is trial 1 with value: 0.8503842672776106.
[I 2025-06-03 16:47:57,288] Trial 2 finished with value: 0.8402549173309771 and parameters: {'n_estimators': 50, 'learning_rate': 0.04816659314916038}. Best is trial 1 with value: 0.8503842672776106.
[I 2025-06-03 16:47:57,444] Trial 3 finished with value: 0.8432880533585634 and parameters: {'n_estimators': 50, 'learning_rate': 0.08055152497412305}. Best is trial 1 with value: 0.8503842672776106.
[I 2025-06-03 16:47:57,743] Trial 4 finished with value: 0.8493370986281126 and parameters: {'n_estimators': 100, 'learning_rate': 0.07558713465340919}. Best is trial 1 with value: 0.8503842672776106.
[I 2025-06-03 16:47:57,929] Trial 5 finished with value: 0.8426033802618667 and parameters: {'n_estimators': 60, 'learning_rate': 0.04622227613883457}. Best is trial 1 with value: 0.8503842672776106

*Best hyperparameters (ab)* {'n_estimators': 100, 'learning_rate': 0.09976662828086057}


...

Cross-validation loop::  33%|███▎      | 1/3 [00:11<00:23, 11.52s/it]

- Omitting rows with missing values: 6042 rows left out


Cross-validation loop::  67%|██████▋   | 2/3 [00:12<00:05,  5.17s/it]

- Omitting rows with missing values: 6042 rows left out


Cross-validation loop:: 100%|██████████| 3/3 [00:13<00:00,  4.35s/it]


In [49]:
experiment.results

{'ab': {0: {'accuracy': 0.93849,
   'brier': 0.17973,
   'f1': 0.25719,
   'precision': 0.62044,
   'recall': 0.16221,
   'h_measure': 0.38153,
   'aucroc': 0.83512,
   'aucpr': 0.36136},
  1: {'accuracy': 0.9318,
   'brier': 0.18276,
   'f1': 0.22695,
   'precision': 0.64,
   'recall': 0.13793,
   'h_measure': 0.40586,
   'aucroc': 0.83511,
   'aucpr': 0.38395},
  2: {'accuracy': 0.93913,
   'brier': 0.18169,
   'f1': 0.21865,
   'precision': 0.61818,
   'recall': 0.13281,
   'h_measure': 0.41818,
   'aucroc': 0.85675,
   'aucpr': 0.38652}},
 'lr': {0: {'accuracy': 0.93436,
   'brier': 0.05806,
   'f1': 0.03676,
   'precision': 0.5,
   'recall': 0.01908,
   'h_measure': 0.17251,
   'aucroc': 0.69408,
   'aucpr': 0.19988},
  1: {'accuracy': 0.93005,
   'brier': 0.05925,
   'f1': 0.1169,
   'precision': 0.69811,
   'recall': 0.06379,
   'h_measure': 0.31217,
   'aucroc': 0.78398,
   'aucpr': 0.32513},
  2: {'accuracy': 0.93625,
   'brier': 0.05659,
   'f1': 0.05566,
   'precision': 0.55

In [14]:

# Todo: implement method that writes experiment.results to a csv file / connects with WandB / ...

# Quick and dirty examplary code:

import pandas as pd

# Convert dictionary of dictionaries to a list of rows
rows = []
for model_name, splits_dict in experiment.results.items():
    for split_idx, metrics_dict in splits_dict.items():
        # Create a row dict with model and split identifiers
        row = {
            'model': model_name,
            'split': split_idx,
        }
        # Update with all metrics in that split
        row.update(metrics_dict)

        rows.append(row)

# Convert the list of row dictionaries into a DataFrame
df = pd.DataFrame(rows)

df

# Summarise df by model (along split) and get the mean and sd of all the metrics, discarding the split and sorting by mean of aucroc
df_summary = df.groupby('model').agg(['mean', 'std']).reset_index()
# df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('split')]
df_summary




model split       accuracy               brier                  f1  \
           mean  std      mean       std      mean       std      mean   
0     ann   1.0  1.0  0.932550  0.004506  0.062403  0.003812  0.000000   
1      dt   1.0  1.0  0.924703  0.002422  0.069583  0.002620  0.294327   
2     knn   1.0  1.0  0.926080  0.003625  0.073033  0.004091  0.080363   
3     lda   1.0  1.0  0.931507  0.003544  0.059013  0.002887  0.126527   
4      lr   1.0  1.0  0.932550  0.004506  0.062073  0.003929  0.000000   
5      rf   1.0  1.0  0.934263  0.003703  0.052610  0.003113  0.262270   
6     svm   1.0  1.0  0.932843  0.004464  0.061293  0.001939  0.021743   
7  tabpfn   1.0  1.0  0.936470  0.003940  0.050707  0.003045  0.293770   
8     xgb   1.0  1.0  0.932550  0.004506  0.058713  0.003197  0.000000   

            precision              recall           h_measure            \
        std      mean       std      mean       std      mean       std   
0  0.000000  0.000000  0.000000  0.000000  0.000000  0.054427  0.010668   
1  0.013192  0.401040  0.021259  0.233630  0.021659  0.232697  0.010765   
2  0.019890  0.247737  0.015965  0.048247  0.013428  0.042327  0.011653   
3  0.025267  0.451177  0.044127  0.073877  0.016343  0.198710  0.018246   
4  0.000000  0.000000  0.000000  0.000000  0.000000  0.066383  0.012676   
5  0.027888  0.541087  0.023853  0.174070  0.026391  0.368213  0.006976   
6  0.005562  0.658273  0.132140  0.011080  0.002904  0.356070  0.016592   
7  0.043818  0.589647  0.030372  0.197573  0.040826  0.402200  0.014832   
8  0.000000  0.000000  0.000000  0.000000  0.000000  0.377150  0.005420   

     aucroc               aucpr            
       mean       std      mean       std  
0  0.621487  0.009011  0.104617  0.006050  
1  0.657687  0.007196  0.215847  0.013250  
2  0.570407  0.012304  0.091307  0.001651  
3  0.684440  0.012835  0.221907  0.024543  
4  0.633873  0.010671  0.116983  0.007833  
5  0.827767  0.006686  0.342080  0.009941  
6  0.800450  0.007351  0.348927  0.010933  
7  0.843273  0.006407  0.386983  0.011910  
8  0.822090  0.007808  0.334273  0.005463